In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.datasets import mnist
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold

import torch.nn.functional as TF
import torch.optim as optim
import os
import math
import matplotlib.pyplot as plt
import pickle

torch.set_num_threads(1)


# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')

In [55]:
data = np.loadtxt('data/iyer/iyer.txt')[1:]
flag_line = data[:,1]
# print(flag_line)
neg1_list = []
for i in range(len(flag_line)):
    if (data[i,1] == -1):
        neg1_list.append(i)
data = np.delete(data, neg1_list, axis=0)

all_x = data[:,2:]
all_y = data[:,1] - 1

# all_x = (all_x - np.min(all_x)) / (np.max(all_x) - np.min(all_x))


all_idx = np.arange(len(all_x))
SEED_train_test = 1
np.random.seed(SEED_train_test)
np.random.shuffle(all_idx)
tr_len= 400
x_tr = all_x[all_idx[:tr_len]]
y_tr = all_y[all_idx[:tr_len]]

x_te = all_x[all_idx[tr_len:]]
y_te = all_y[all_idx[tr_len:]]

print(x_tr.shape)
print(x_te.shape)

(400, 12)
(84, 12)


In [56]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32,kernel_size=5,stride=1)
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(32)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=128, kernel_size=5, stride=1)
        self.bn2 = nn.BatchNorm1d(128)
#         self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1)
#         self.bn3 = nn.BatchNorm1d(256)
        self.fc1 = nn.Linear(4*128,64)
#         self.fc2 = nn.Linear(64,64)
        self.fc2 = nn.Linear(64,10)
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.bn1(x)
        x = self.relu(self.conv2(x))
        x = self.bn2(x)
#         x = self.relu(self.conv3(x))
#         x = self.bn3(x)
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
#         x = self.relu(self.fc2(x))
        x = self.fc2(x)
        return x

In [57]:
LOG_DIR = 'Logs/'
SAVE_DIR = 'Models/'
K_FOLD = 5
kf = KFold(n_splits=K_FOLD)
k_idx = 0
for train_idx, val_idx in kf.split(x_tr):
#     print("%s %s" % (train, test))
#     train_x = torch.from_numpy(x_tr[train_idx]).unsqueeze(1)
    train_x = torch.Tensor(list(x_tr[train_idx])).unsqueeze(1)
    train_label = torch.Tensor(list(y_tr[train_idx]))
    val_x = torch.Tensor(list(x_tr[val_idx])).unsqueeze(1)
    val_label =torch.Tensor(list(y_tr[val_idx]))
#     lr_step = [20,50]
    model = CNN().double().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9,weight_decay=5e-4)
#     train_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, lr_step, gamma=0.1)
    
    cost = nn.CrossEntropyLoss()
    epoch = 100
    train_loss = []
    val_loss = []
    LEARN_SEED = 42
    torch.manual_seed(LEARN_SEED)
    best_val_acc = 0.0
    for _epoch in range(epoch):
        train_x, train_label = train_x.double().to(device), train_label.to(device).long()
        optimizer.zero_grad()
        outputs = model(train_x)
        loss = cost(outputs, train_label)
        loss.backward()
        optimizer.step()
#         train_scheduler.step()
        print('Epoch:%d, loss:%.6f'%(_epoch, loss.sum().item()))
        train_loss.append(loss.sum().item())
        
        correct = 0
        _sum = 0

        val_x, val_label = val_x.double().to(device), val_label.to(device).long()
        outputs = model(val_x).detach()
        t_loss = cost(outputs, val_label)
        predict_ys = torch.argmax(outputs, axis=-1)
        _ = predict_ys.detach().data == val_label
        correct += torch.sum(_, axis=-1)
        _sum += _.shape[0]
        val_loss.append(t_loss.sum().item())
        
        val_acc = 100*correct / _sum
        print('Validation accuracy: %.4f'%val_acc)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_ckpt = {'net':model.state_dict(),
                        'optim':optimizer.state_dict(),
                        'epoch':_epoch,
                        'val_acc':best_val_acc}
            best_save_path = SAVE_DIR + "IYER_CNN_Val_K_%d_model"%k_idx
            torch.save(best_ckpt, best_save_path)
            
    log_save_path = LOG_DIR + "IYER_CNN_Val_SEED_%d_log"%k_idx
    pickle.dump([train_loss, val_loss], open(log_save_path,"wb"))
    
    k_idx += 1

Epoch:0, loss:2.313606
Validation accuracy: 7.5000
Epoch:1, loss:2.278782
Validation accuracy: 10.0000
Epoch:2, loss:2.214592
Validation accuracy: 30.0000
Epoch:3, loss:2.128767
Validation accuracy: 31.2500
Epoch:4, loss:2.028935
Validation accuracy: 31.2500
Epoch:5, loss:1.918315
Validation accuracy: 32.5000
Epoch:6, loss:1.799595
Validation accuracy: 33.7500
Epoch:7, loss:1.676979
Validation accuracy: 46.2500
Epoch:8, loss:1.556669
Validation accuracy: 48.7500
Epoch:9, loss:1.444824
Validation accuracy: 51.2500
Epoch:10, loss:1.348049
Validation accuracy: 50.0000
Epoch:11, loss:1.269709
Validation accuracy: 50.0000
Epoch:12, loss:1.208162
Validation accuracy: 51.2500
Epoch:13, loss:1.157815
Validation accuracy: 52.5000
Epoch:14, loss:1.114095
Validation accuracy: 53.7500
Epoch:15, loss:1.075343
Validation accuracy: 57.5000
Epoch:16, loss:1.042770
Validation accuracy: 61.2500
Epoch:17, loss:1.017553
Validation accuracy: 58.7500
Epoch:18, loss:0.995605
Validation accuracy: 57.5000
Epoc

Validation accuracy: 80.0000
Epoch:57, loss:0.323535
Validation accuracy: 80.0000
Epoch:58, loss:0.316443
Validation accuracy: 80.0000
Epoch:59, loss:0.310292
Validation accuracy: 80.0000
Epoch:60, loss:0.304093
Validation accuracy: 78.7500
Epoch:61, loss:0.297491
Validation accuracy: 77.5000
Epoch:62, loss:0.291268
Validation accuracy: 76.2500
Epoch:63, loss:0.285832
Validation accuracy: 76.2500
Epoch:64, loss:0.280496
Validation accuracy: 76.2500
Epoch:65, loss:0.275022
Validation accuracy: 76.2500
Epoch:66, loss:0.269748
Validation accuracy: 76.2500
Epoch:67, loss:0.264757
Validation accuracy: 76.2500
Epoch:68, loss:0.259969
Validation accuracy: 77.5000
Epoch:69, loss:0.255199
Validation accuracy: 77.5000
Epoch:70, loss:0.250513
Validation accuracy: 77.5000
Epoch:71, loss:0.245937
Validation accuracy: 77.5000
Epoch:72, loss:0.241707
Validation accuracy: 77.5000
Epoch:73, loss:0.237819
Validation accuracy: 77.5000
Epoch:74, loss:0.233927
Validation accuracy: 77.5000
Epoch:75, loss:0.

Validation accuracy: 48.7500
Epoch:13, loss:1.433387
Validation accuracy: 50.0000
Epoch:14, loss:1.357969
Validation accuracy: 50.0000
Epoch:15, loss:1.291717
Validation accuracy: 50.0000
Epoch:16, loss:1.233475
Validation accuracy: 50.0000
Epoch:17, loss:1.179491
Validation accuracy: 50.0000
Epoch:18, loss:1.126625
Validation accuracy: 53.7500
Epoch:19, loss:1.075416
Validation accuracy: 55.0000
Epoch:20, loss:1.029727
Validation accuracy: 60.0000
Epoch:21, loss:0.993716
Validation accuracy: 63.7500
Epoch:22, loss:0.967782
Validation accuracy: 65.0000
Epoch:23, loss:0.945639
Validation accuracy: 70.0000
Epoch:24, loss:0.918654
Validation accuracy: 65.0000
Epoch:25, loss:0.883681
Validation accuracy: 63.7500
Epoch:26, loss:0.846338
Validation accuracy: 62.5000
Epoch:27, loss:0.814989
Validation accuracy: 61.2500
Epoch:28, loss:0.792375
Validation accuracy: 61.2500
Epoch:29, loss:0.773617
Validation accuracy: 63.7500
Epoch:30, loss:0.752282
Validation accuracy: 65.0000
Epoch:31, loss:0.

Epoch:70, loss:0.300379
Validation accuracy: 82.5000
Epoch:71, loss:0.295617
Validation accuracy: 83.7500
Epoch:72, loss:0.290925
Validation accuracy: 83.7500
Epoch:73, loss:0.286477
Validation accuracy: 83.7500
Epoch:74, loss:0.282056
Validation accuracy: 83.7500
Epoch:75, loss:0.277743
Validation accuracy: 83.7500
Epoch:76, loss:0.273668
Validation accuracy: 83.7500
Epoch:77, loss:0.269544
Validation accuracy: 83.7500
Epoch:78, loss:0.265486
Validation accuracy: 83.7500
Epoch:79, loss:0.261540
Validation accuracy: 83.7500
Epoch:80, loss:0.257561
Validation accuracy: 83.7500
Epoch:81, loss:0.253804
Validation accuracy: 83.7500
Epoch:82, loss:0.250076
Validation accuracy: 83.7500
Epoch:83, loss:0.246380
Validation accuracy: 83.7500
Epoch:84, loss:0.242798
Validation accuracy: 83.7500
Epoch:85, loss:0.239220
Validation accuracy: 83.7500
Epoch:86, loss:0.235685
Validation accuracy: 83.7500
Epoch:87, loss:0.232235
Validation accuracy: 83.7500
Epoch:88, loss:0.228809
Validation accuracy: 8

In [58]:
LOG_DIR = 'Logs/'
SAVE_DIR = 'Models/'
K_FOLD = 5
kf = KFold(n_splits=K_FOLD)
k_idx = 0
for train_idx, val_idx in kf.split(x_tr):
    val_x = x_tr[val_idx]
    y_vals = y_tr[val_idx]
    
    y_vals_onehot = np.eye(10)[y_vals.astype(int)].reshape([-1,1]).reshape([-1,10])
    val_x = torch.Tensor(list(val_x)).unsqueeze(1)
    val_label =torch.Tensor(list(y_vals))
    
    model = CNN().double().to(device)
    best_save_path = SAVE_DIR + "IYER_CNN_Val_K_%d_model"%k_idx
    model.load_state_dict(torch.load(best_save_path)['net'])
    
    val_x, val_label = val_x.double().to(device), val_label.to(device).long()
       
    outputs = model(val_x).detach()
    y_output = TF.softmax(outputs,-1)
    y_outputs = y_output.detach().cpu().data.numpy()
    print(y_outputs.shape)
    y_pred = torch.argmax(outputs, axis=-1)
    y_preds = y_pred.detach().cpu().data.numpy()
        
    y_outputs = np.array(y_outputs).reshape([-1,10])
    print(metrics.accuracy_score(y_vals,y_preds))
    print(metrics.f1_score(y_vals,y_preds,average='weighted'))
    print(metrics.roc_auc_score(y_vals_onehot,y_outputs,average='samples',multi_class='ovo'))
    k_idx += 1
    

(80, 10)
0.8
0.800229723979724
0.9694444444444447
(80, 10)
0.825
0.7916462454979897
0.9708333333333332
(80, 10)
0.8375
0.8292512513904338
0.9777777777777779
(80, 10)
0.775
0.7591071428571429
0.9680555555555557
(80, 10)
0.85
0.8471821674196425
0.9833333333333332


In [59]:
LOG_DIR = 'Logs/'
SAVE_DIR = 'Models/'

k_idx = 4
    
y_te_onehot = np.eye(10)[y_te.astype(int)].reshape([-1,1]).reshape([-1,10])

te_x = torch.Tensor(list(x_te)).unsqueeze(1)
te_label =torch.Tensor(list(y_te))

model = CNN().double().to(device)
best_save_path = SAVE_DIR + "IYER_CNN_Val_K_%d_model"%k_idx
model.load_state_dict(torch.load(best_save_path)['net'])

te_x, te_label = te_x.double().to(device), te_label.to(device).long()

outputs = model(te_x).detach()
y_output = TF.softmax(outputs,-1)
y_outputs = y_output.detach().cpu().data.numpy()
y_pred = torch.argmax(outputs, axis=-1)
y_preds = y_pred.detach().cpu().data.numpy()

y_outputs = np.array(y_outputs).reshape([-1,10])
print(metrics.accuracy_score(y_te,y_preds))
print(metrics.f1_score(y_te,y_preds,average='weighted'))
print(metrics.roc_auc_score(y_te_onehot,y_outputs,average='samples',multi_class='ovo'))

    

0.8809523809523809
0.8811595728137082
0.9867724867724867
